<a href="https://colab.research.google.com/github/bayazidHossain2/ImageProcessingEasyExam/blob/main/Imagetrain%26testFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import os
import random
import numpy as np 
import matplotlib.pyplot as plt 

# img = cv2.imread("img/test.jpg")
# img = cv2.resize(img,(625,810))
# plt.imshow(img)
# print(img.shape)


In [2]:
dir = r"img"
catagories = ['bbox','fbox']

In [3]:
data = list()
for catagory in catagories:
  folder = os.path.join(dir,catagory)
  print(folder)
  label = catagories.index(catagory)

  for img in os.listdir(folder):
    img = os.path.join(folder,img)
    #print(img)
    img_arr = cv2.imread(img)
    img_arr = cv2.resize(img_arr,(60,25))
    data.append([img_arr,label])


img/bbox
img/fbox


In [4]:
random.shuffle(data)


In [7]:
x = list()
y = list()
for features,label in data:
  x.append(features)
  y.append(label)

print(len(y))

420


In [ ]:
x = np.array(x)
y = np.array(y)
# print(x)

In [9]:
x = x/255
#print(x)

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Activation

In [22]:
model = Sequential()
model.add(Conv2D(30,(3,3),input_shape=x.shape[1:],activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(20,(3,3),input_shape=x.shape[1:],activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(10,(3,3),input_shape=x.shape[1:],activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(2,activation='softmax'))


In [23]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [24]:
model.fit(x,y,epochs=15,validation_split=0.1)

Epoch 1/15
12/12 [==============================] - 1s 42ms/step - loss: 0.6709 - accuracy: 0.5979 - val_loss: 0.6488 - val_accuracy: 0.6429
Epoch 2/15
12/12 [==============================] - 0s 33ms/step - loss: 0.5843 - accuracy: 0.7222 - val_loss: 0.6627 - val_accuracy: 0.6429
Epoch 3/15
12/12 [==============================] - 0s 32ms/step - loss: 0.5632 - accuracy: 0.7222 - val_loss: 0.6220 - val_accuracy: 0.6429
Epoch 4/15
12/12 [==============================] - 0s 32ms/step - loss: 0.5394 - accuracy: 0.7222 - val_loss: 0.6066 - val_accuracy: 0.6429
Epoch 5/15
12/12 [==============================] - 0s 32ms/step - loss: 0.5085 - accuracy: 0.7222 - val_loss: 0.5424 - val_accuracy: 0.6429
Epoch 6/15
12/12 [==============================] - 0s 32ms/step - loss: 0.4723 - accuracy: 0.7222 - val_loss: 0.5594 - val_accuracy: 0.6429
Epoch 7/15
12/12 [==============================] - 0s 34ms/step - loss: 0.4476 - accuracy: 0.7884 - val_loss: 0.4282 - val_accuracy: 0.7857
Epoch 8/15
12

In [25]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 23, 58, 30)        840       
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 11, 29, 30)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 9, 27, 20)         5420      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 4, 13, 20)        0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 2, 11, 10)         1810      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 1, 5, 10)        

In [27]:
tf.saved_model.save(model,"saved_model_keras_dir")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('checkMarkDetectorModel.tflite','wb') as f:
  f.write(tflite_model)

In [ ]:
from keras.preprocessing import image
import numpy as np
c = 0
for i in range(5):
  for j in range(4):
    st = '2ci '+str(i)+' '+str(j)+'.jpg'
    print('reading :',st)
    img_pred = cv2.imread(r"img/test/"+st)
    img_pred = cv2.resize(img_pred,(60,25))
    img_pred = np.expand_dims(img_pred,axis=0)
    img_pred = np.array(img_pred)
    res = model.predict(img_pred)
    print(res)

    if res[0][0] > res[0][1]:
      print('Filled')
    else:
      print('Blanked')

reading : 2ci 0 0.jpg
1/1 [==============================] - 0s 20ms/step
[[7.321319e-06 9.999927e-01]]
Filled
reading : 2ci 0 1.jpg
1/1 [==============================] - 0s 33ms/step
[[4.3291777e-07 9.9999952e-01]]
Filled
reading : 2ci 0 2.jpg
1/1 [==============================] - 0s 21ms/step
[[4.8150848e-09 1.0000000e+00]]
Filled
reading : 2ci 0 3.jpg
1/1 [==============================] - 0s 20ms/step
[[2.0872490e-05 9.9997914e-01]]
Filled
reading : 2ci 1 0.jpg
1/1 [==============================] - 0s 21ms/step
[[1.6175235e-04 9.9983823e-01]]
Filled
reading : 2ci 1 1.jpg
1/1 [==============================] - 0s 25ms/step
[[1.6360003e-05 9.9998367e-01]]
Filled
reading : 2ci 1 2.jpg
1/1 [==============================] - 0s 22ms/step
[[6.241526e-06 9.999938e-01]]
Filled
reading : 2ci 1 3.jpg
1/1 [==============================] - 0s 22ms/step
[[1.3065775e-04 9.9986935e-01]]
Filled
reading : 2ci 2 0.jpg
1/1 [==============================] - 0s 31ms/step
[[6.4942396e-05 9.9993503e

In [ ]:
from keras.preprocessing import image
import numpy as np
st = '1ci 0 3.jpg'
print('reading :',st)
img_pred = cv2.imread(r"img/test/"+st)
img_pred = cv2.resize(img_pred,(60,25))
img_pred = np.expand_dims(img_pred,axis=0)
img_pred = np.array(img_pred)
res = model.predict(img_pred)
print(res)

if res[0][0] > res[0][1]:
  print('Filled')
else:
  print('Blanked')

reading : 1ci 0 3.jpg
1/1 [==============================] - 0s 21ms/step
[[9.9995506e-01 4.4911369e-05]]
Filled
